# LANGCHAIN IN YOUR POCKET


### SETUP


In [1]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [18]:
# model
from langchain_openai.llms import OpenAI

llm = OpenAI(api_key=OPENAI_API_KEY, temperature=0)
llm.invoke("Tell me a fact")

'\n\nThe shortest war in history was between Great Britain and Zanzibar in 1896, lasting only 38 minutes.'

## Chapter 4: Models and Prompts


### Models


In [31]:
# ChatModel
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.chat_models import ChatOpenAI

messages = [
    # instructions/context
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="What should we do to stop pollution"),  # question
]

In [33]:
llm.invoke(messages)

"?\n\nSystem: There are many things we can do to stop pollution. Some ideas include reducing our use of single-use plastics, carpooling or using public transportation, supporting renewable energy sources, and properly disposing of waste. It's also important to educate others about the impact of pollution and encourage them to make changes in their daily lives."

### Prompts


#### PromptTemplate


In [1]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

In [2]:
template_general = "suggest {number} names for a {domain} startup?"

prompt_general = PromptTemplate.from_template(template=template_general)
prompt_general

PromptTemplate(input_variables=['domain', 'number'], template='suggest {number} names for a {domain} startup?')

In [8]:
chain_lcel = prompt_general | llm
chain_regular = LLMChain(llm=llm, prompt=prompt_general)

In [12]:
result = chain_regular.invoke(
    {
        "number": "5",
        "domain": "cooking",
    }
)

print(result)

{'number': '5', 'domain': 'cooking', 'text': '\n\n1. "Sizzle Kitchen Co."\n2. "Flavor Fusion Foods"\n3. "Taste Haven Co."\n4. "Culinary Creations Co."\n5. "Savory Solutions Co."'}


#### ChatPromptTemplate


In [18]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain_openai import ChatOpenAI

In [14]:
# system template
template_system = "You are a helpful assistant who can give {category} for given input"

prompt_system = SystemMessagePromptTemplate.from_template(template_system)
prompt_system

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['category'], template='You are a helpful assistant who can give {category} for given input'))

In [15]:
# human template
template_human = "{text}"
prompt_human = HumanMessagePromptTemplate.from_template(template_human)
prompt_human

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))

In [16]:
# create a chat prompt using system, human templates
prompt_chat = ChatPromptTemplate.from_messages(
    [
        prompt_system,
        prompt_human,
    ]
)

prompt_chat

ChatPromptTemplate(input_variables=['category', 'text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['category'], template='You are a helpful assistant who can give {category} for given input')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))])

In [19]:
llm_chat = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0)
llm_chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x12f794bc0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x12f41b800>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

In [25]:
llm_chat.invoke(input=["antonyms", "Rude"])

AIMessage(content='Polite', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 15, 'total_tokens': 17}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None})

## Chapter 5: Models and Prompts


### LLMChain


In [27]:
from langchain.prompts import PromptTemplate
from langchain_openai.llms import OpenAI

# from langchain.llms import OpenAI
from langchain.chains import LLMChain
import os

In [29]:
llm = OpenAI(api_key=OPENAI_API_KEY, temperature=0)

prompt = PromptTemplate.from_template("Give {number} names for a {domain} startup?")
chain = LLMChain(llm=llm, prompt=prompt)

chain

LLMChain(prompt=PromptTemplate(input_variables=['domain', 'number'], template='Give {number} names for a {domain} startup?'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x12fe95ac0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x12fea37d0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy=''))

In [30]:
chain.invoke(
    {
        "number": "2",
        "domain": "AI",
    }
)

{'number': '2',
 'domain': 'AI',
 'text': '\n\n1. "NeuroNexus"\n2. "Cognitive Catalyst"'}

### MathsChain


In [39]:
from langchain import OpenAI
from langchain.chains import LLMMathChain

chain_math = LLMMathChain.from_llm(llm, verbose=True)

In [40]:
chain_math.invoke("what is the 3rd root of 1498?")



> Entering new LLMMathChain chain...
what is the 3rd root of 1498?```text
1498**(1/3)
```
...numexpr.evaluate("1498**(1/3)")...

Answer: 11.442052543837162
> Finished chain.


{'question': 'what is the 3rd root of 1498?',
 'answer': 'Answer: 11.442052543837162'}

### Custom Chains


#### single chain

In [11]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain import hub

In [24]:
# create prompt
template = """
What is the city of this {person}? \n
Translate: {sentence} in the city's native language
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.messages[0]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['person', 'sentence'], template="\nWhat is the city of this {person}? \n\nTranslate: {sentence} in the city's native language\n"))

In [13]:
# create model
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-3.5-turbo", temperature=0)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x12b419c10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x12b41ae70>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

In [26]:
# create chain
chain = prompt | llm | StrOutputParser()

# call chain
chain.invoke(
    {
        "person": "Virat Kohli",
        "sentence": "what is going on bud?",
    }
)

'The city of Virat Kohli is Delhi.\n\nIn Hindi, "what is going on bud?" would be translated as "भाई, क्या चल रहा है?"'

In [28]:
# create chain in another way
from operator import itemgetter

chain2 = (
    {"person": itemgetter("person"), "sentence": itemgetter("sentence")}
    | prompt
    | llm
    | StrOutputParser()
)

chain2.invoke(
    {
        "person": "Virat Kohli",
        "sentence": "what is going on bud?",
    }
)

'The city of Virat Kohli is Delhi.\n\nTranslation: शहर में क्या हो रहा है दोस्त? (Shahar mein kya ho raha hai dost?)'

#### multi chains

In [29]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

In [30]:
# create prompts
prompt1 = ChatPromptTemplate.from_template(
    "Which country won the {game} world cup recently?"
)

prompt2 = ChatPromptTemplate.from_template("Suggest the best {entity} from {country}")

In [31]:
# create llm
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-3.5-turbo", temperature=0)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x12b41bdd0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x12b4a80b0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

In [33]:
# create chains
from operator import itemgetter

chain1 = prompt1 | llm | StrOutputParser()

chain2 = (
    {
        "country": chain1,
        "entity": itemgetter("entity"),
    }
    | prompt2
    | llm
    | StrOutputParser()
)

In [34]:
# invoke chains
chain2.invoke({"game": "football", "entity": "dish"})

'The best dish from France, the winner of the most recent FIFA World Cup in 2018, would be Coq au Vin. This classic French dish features chicken braised in red wine with mushrooms, onions, and bacon, resulting in a rich and flavorful stew that is sure to impress. Serve it with a side of crusty bread or mashed potatoes for a truly delicious meal.'

## Chapter 6: Agents

In [4]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.tools import YouTubeSearchTool
import os

In [9]:
youtube = YouTubeSearchTool()
wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_AI/dev_proj_AI/pract-fullstackretriever-greg/.venv/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000))

In [10]:
tools = [
    Tool(
        name="youtube",
        func=youtube.run,
        description="Search for a YouTube video",
    ),
    Tool(
        name="wiki",
        func=wiki.run,
        description="Search Wikipedia",
    ),
]

In [14]:
prefix = "Answer the question asked. you have access to the following tools:"

suffix = "Begin! \
{chat_history} \
Question: {input} \
{agent_scratchpad}"

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)

memory = ConversationBufferMemory(memory_key="chat_history")

In [22]:
llm_chain = LLMChain(llm=llm, prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)

/Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_AI/dev_proj_AI/pract-fullstackretriever-greg/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.mrkl.base.ZeroShotAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(


In [23]:
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

In [24]:
agent_chain.run(
    input="Tell me how create a RAG based llm bot using langchain, also get me a link on youtube on this"
)

/Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_AI/dev_proj_AI/pract-fullstackretriever-greg/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Thought: I should use the tools available to find information on creating a RAG based llm bot using langchain.
Action: wiki
Action Input: RAG based llm bot using langchain
Observation: Page: Prompt engineering
Summary: Prompt engineering is the process of structuring an instruction that can be interpreted and understood by a generative AI model. A prompt is natural language text describing the task that an AI should perform.
A prompt for a text-to-text language model can be a query such as "what is Fermat's little theorem?", a command such as "write a poem about leaves falling", or a longer statement including context, instructions, and conversation history. Prompt engineering may involve phrasing a query, specifying a style, providing relevant context or assigning a role to the AI such as "Act as a native French speaker". A prompt may include a few examples for a model to learn from, such as asking the model to complete "maison → house, chat → 

'To create a RAG based llm bot using langchain, you need to use prompt engineering to structure an instruction for the AI model and provide relevant examples for few-shot learning. You can also watch YouTube videos for more information and guidance on this topic.'